# Pyrcrack on a python notebook

This is a simple demonstration of pyrcrack being used in a python notebook. 

This allows for a powerful web interface, with saved states and extensible, useful for everyday penetration testing tasks.

### Extracting available wifi interfaces

First, we're going to use airmon to get the available wireless interfaces and its drivers.

In [36]:
import pyrcrack

airmon = pyrcrack.AirmonNg()

[a.asdict() for a in await airmon.interfaces]

[{'phy': 'phy1',
  'interface': 'wlp3s0',
  'driver': 'iwlwifi',
  'chipset': 'Intel Corporation Wireless 7260 (rev 83)'}]

### Scanning

Now we'll use airmon-ng, without bounding it to any access point or channel so it scans freely

In [26]:
async with airmon("wlp3s0") as mon:
    async with pyrcrack.AirodumpNg() as pdump:
        async for aps in pdump(mon.monitor_interface):
            break # So notebook execution doesn't get stuck here 
aps

[TESTNET - (FF:FF:FF:FF:FF:FF)]

### Now, select just THAT specific AP

The airodump property of any ap will return the required parameters for airodump-ng to select that client.

In [23]:
aps[0].airodump

{'channel': '11', 'bssid': 'FF:FF:FF:FF:FF:FF'}

In [37]:
async with airmon("wlp3s0") as mon:
    # Re-set the interface in monitor mode as in 
    # previous execution it would have been cleaned up
    async with pyrcrack.AirodumpNg() as pdump:
        async for result in pdump(mon.monitor_interface, **aps[0].airodump):
            break
            
[a.asdict() for a in result]

[{'essid': 'TESTNET',
  'bssid': 'FF:FF:FF:FF:FF:FF',
  'packets': '2',
  'dbm': '-74',
  'score': '76',
  'channel': '1',
  'encryption': 'WPA+PSK/WPA+AES-CCM'}]